In [9]:
%load_ext autoreload
%autoreload 2

from models.ranked_transformer import Moonshot
import torch, pytorch_lightning as pl

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
from models.chemformer.tokeniser import MolEncTokeniser
from models.chemformer.utils import REGEX


path = "tempdata/chemformer/ms_e6.ckpt"
token_file = "tempdata/chemformer/bart_vocab.txt"

tokeniser = MolEncTokeniser.from_vocab_file(
  token_file, REGEX, 272
)
model = Moonshot.load_from_checkpoint(path, tokeniser=tokeniser).cuda()


Initialized SignCoordinateEncoder[512] with dims [224, 224, 64] and 2 positional encoders. 64 bits are reserved for encoding the final bit





Ranked Transformer callin super


In [11]:
from utils.init_utils.data_init import data_mux


feats = ["HSQC", "SMILES"]
feat_handlers = ["pad", "tokenise"]
ds_path = "tempdata/SMILES_dataset/"

gim = data_mux(feats, feat_handlers, ds_path, token_file = token_file)

In [12]:
gim.setup("fit")
val_dl = gim.val_dataloader()

In [13]:
from rdkit import Chem, RDLogger
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Draw

RDLogger.DisableLog("rdApp.*")

In [21]:
ct = 0
choice = ""
import random


pl.utilities.seed.isolate_rng(1234)

for batch in val_dl:
  hsqc, collated_smiles = batch
  hsqc = hsqc.cuda()
  collated_smiles = {k:(v if type(v) is not torch.Tensor else v.cuda()) for k,v in collated_smiles.items()}
  out = model.sample_rdm(hsqc, temperature=1e-6, gen_len = 128)
  raw = collated_smiles["raw_smiles"]
  output_strs = out["mol_strs"]
  print(len(output_strs))
  for idx, (r, o) in enumerate(zip(raw, output_strs)):
    print(torch.sum(hsqc[idx]), Chem.MolFromSmiles(o) is not None, len(o), r, o)
  ct += 1
  if ct == 1:
    break

Max seq len: 128


100%|██████████| 127/127 [00:02<00:00, 54.01it/s]


32
tensor(9239.9336, device='cuda:0') False 127 c12ccc(O)c(O)c1-c1cc(O)cc3c1C(C2)N(C)CC3.Br ccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccc
tensor(25262.3242, device='cuda:0') False 127 c12cc[nH]c1cc(Br)cc2Br ccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccc
tensor(47867.4492, device='cuda:0') False 127 c12c(cncc1)[nH]c1ccc(Br)cc21 ccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccc
tensor(35896.0391, device='cuda:0') False 127 c12c([nH]cc1)ccc(Br)c2 ccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccc
tensor(-1252.0664, device='cuda:0') True 127 C(#CC#C)C#CC=CCO CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC

In [60]:
v = Chem.MolFromSmiles("(CCcCOCc4CCCCCC(5cCCC=CCCc2CCCC)=1C=C=CC)")
print(v)

None
